* 给姚媛整理的疾病表格式  
* 根据姚媛的整理结果更新疾病库  
* 根据姚媛整理的结果更新疾病库科室

In [21]:
import sys
sys.path.append('/home/ian/code/github')

In [24]:
from utils1.pymongo_wrapper import pymongo_wrapper
pmw = pymongo_wrapper()
table = pmw.get_collection('jiankang39','diseases')

In [9]:
c=list(pmw.find_all(table,fieldlist=['疾病名称','简介','挂号的科室'],conditions={'罕见程度':{'$ne':0}}))

In [10]:
len(c)

6483

In [11]:
from utils1.pandas_wrapper import pandas_wrapper as pdw

In [14]:
df = pdw.build_df_from_dict({'疾病名称':[i['疾病名称'] for i in c],'简介':[i['简介'] for i in c],'挂号的科室':[i['挂号的科室'] if '挂号的科室' in i else '' for i in c]})
df.head()

,疾病名称,简介,挂号的科室
0,偏头痛,偏头痛是一类有家族发病倾向的周期性发作疾病。表现为发作性的偏侧搏动性头痛，伴恶心、呕吐及羞明...,神经内科
1,帕金森,帕金森病(Parkinsons disease)又称震颤麻痹，巴金森氏症或柏金逊症，多在60...,神经内科 中医科
2,老年痴呆症,阿尔茨海默病(Alzheimer’s disease，AD)是慢性进行性中枢神经系统变性病导...,
3,小儿多动症,儿童多动症、多动综合征(hyperkinetic syndrome of childhood...,儿科 精神病科
4,小儿脑瘫,小儿脑瘫也称为儿童脑瘫，病变部位在脑，累及四肢，常伴有癫痫、智力缺陷，行为异常、精神障碍及视...,儿科 神经内科


In [20]:
df.to_csv('keshi.csv',encoding='utf8')

In [12]:
l1 = list()
l2 = list()
l3 = list()
for i in c:
    if '多发人群' in i:
        l1.append(i['多发人群'])
    else:
        l1.append(None)
    if '简介' in i:
        l2.append(i['简介'])
    else:
        l2.append(None)
    if '典型症状' in i:
        l3.append(','.join(i['典型症状']))
    else:
        l3.append(None)

In [17]:
df = pdw.build_df_from_dict({'疾病名称':[i['疾病名称'] for i in c],'简介':l2,'典型症状':l3,'多发人群':l1})
df.head()

NameError: name 'pdw' is not defined

In [15]:
df.to_csv('/home/ian/code/data/daozhen/39疾病列表.csv',encoding='utf8')

## 根据姚媛的整理结果更新疾病库

In [22]:
import pandas as pd

In [18]:
df = pd.read_excel('/home/ian/code/data/daozhen/39疾病列表-更新.xlsx').fillna('')
df.head()

,疾病名称,简介,典型症状,多发人群,罕见程度,多发性别1：男2：女,多发年龄,限定性别,限定年龄,备注,Unnamed: 10
0,偏头痛,偏头痛是一类有家族发病倾向的周期性发作疾病。表现为发作性的偏侧搏动性头痛，伴恶心、呕吐及羞明...,"单侧头痛,电击样痛或刺痛的头痛,头痛为搏动性",女性占2/3以上， 10岁前、20岁前、40岁前发病分别为25%， 55%， 90%。,2,2,,,,,
1,帕金森,帕金森病(Parkinsons disease)又称震颤麻痹，巴金森氏症或柏金逊症，多在60...,"静止性震颤,肌张力过高,感觉障碍",老年人，于40～70岁，60岁后发病率增高，30多岁前发病少见。,2,,67,,,,
2,老年痴呆症,阿尔茨海默病(Alzheimer’s disease，AD)是慢性进行性中枢神经系统变性病导...,,,1,,,,7,,
3,蛛网膜下腔出血,蛛网膜下腔出血(subarachnoid hemorrhage，SAH)是出血性脑血管病的一...,"脑膜刺激症状,定位神经体征,偏瘫",30～40岁青壮年多见，也有的报道80%的发病年龄在30～69岁。男性稍多于女性。,0,1,,,,,
4,小儿多动症,儿童多动症、多动综合征(hyperkinetic syndrome of childhood...,"注意力不集中,情绪起伏大,学习困难",婴幼儿,2,,,,23,,


In [21]:
df.shape

(7450, 11)

In [13]:
otherdict = {'普通':[0],'孕妇':[1],'产褥期':[2],'孕产妇':[1,2]}

In [20]:
for i,n in df['疾病名称'].items():
    try:
        d = table.find_one({'疾病名称':n})
        d['罕见程度'] = int(df.iloc[i,4])
        if df.iloc[i,5]:
            d['多发性别'] = [int(df.iloc[i,5])-1]
        else:
            d['多发性别'] = [0,1]

        if df.iloc[i,6]:
            d['多发年龄'] = list(str(int(df.iloc[i,6])))
        else:
            d['多发年龄'] = ['1', '2', '3', '4', '5', '6', '7']
        if df.iloc[i,7]:
            d['限定性别'] = [int(df.iloc[i,7])-1]
        else:
            d['限定性别'] = [0,1]
        if df.iloc[i,8]:
            d['限定年龄'] = list(str(int(df.iloc[i,8])))
        else:
            d['限定年龄'] = ['1', '2', '3', '4', '5', '6', '7']
        if df.iloc[i,9]:
            if df.iloc[i,9] in otherdict:
                d['备注'] = otherdict[df.iloc[i,9]]
        else:
            d['备注'] = [0,1,2]
        pmw.update_doc(table,{'_id':d['_id']},d)
    except Exception as e:
        print(e)
        print(i,n)

### 根据姚媛整理的解决更新疾病库科室
向疾病库中增加一列：挂号的科室1

In [34]:

df = pd.read_excel('/home/ian/code/data/daozhen/疾病-科室181119.xlsx').fillna('')
df.head()

,简介,挂号的科室,科室1-改动后,科室2-改动后,Unnamed: 4
偏头痛,偏头痛是一类有家族发病倾向的周期性发作疾病。表现为发作性的偏侧搏动性头痛，伴恶心、呕吐及羞明...,神经内科,神经内科,,
帕金森,帕金森病(Parkinsons disease)又称震颤麻痹，巴金森氏症或柏金逊症，多在60...,神经内科 中医科,神经内科,中医科,
老年痴呆症,阿尔茨海默病(Alzheimer’s disease，AD)是慢性进行性中枢神经系统变性病导...,,,,
小儿多动症,儿童多动症、多动综合征(hyperkinetic syndrome of childhood...,儿科 精神病科,儿科,精神科,
小儿脑瘫,小儿脑瘫也称为儿童脑瘫，病变部位在脑，累及四肢，常伴有癫痫、智力缺陷，行为异常、精神障碍及视...,儿科 神经内科,儿科,神经内科,


In [36]:
df1 = df.iloc[:,2:4]
df1.head()

,科室1-改动后,科室2-改动后
偏头痛,神经内科,
帕金森,神经内科,中医科
老年痴呆症,,
小儿多动症,儿科,精神科
小儿脑瘫,儿科,神经内科


In [47]:
for line in df1.itertuples():
    l1 = list()
    if(line[1]):
        t = line[1]
        if not line[1].endswith('科'):
            t = line[1] + '科'
        l1.append(t)
    if(line[2]):
        t = line[2]
        if not line[2].endswith('科'):
            t = line[2] + '科'
        l1.append(t)
    print(l1, line[0])
    if l1:
        pmw.update_doc(table,{'疾病名称':line[0]},{'挂号的科室':l1})

['神经内科'] 偏头痛
['神经内科', '中医科'] 帕金森
[] 老年痴呆症
['儿科', '精神科'] 小儿多动症
['儿科', '神经内科'] 小儿脑瘫
['神经内科', '儿科'] 小儿抽动症
['中医科', '耳鼻喉科'] 脑鸣
['精神科', '神经内科'] 嗜睡症
['神经内科'] 脑萎缩
[] 老年眩晕
['神经内科', '普外科'] 烟雾病
['儿科', '神经内科'] 婴儿痉挛症
['急诊科'] 慢性铅中毒
['神经内科', '精神科'] 抽动障碍
['急诊科', '消化内科'] 安眠药中毒
['神经内科', '脑外科'] 颞叶癫痫
['肿瘤科', '耳鼻喉科'] 听神经瘤
['肿瘤科', '脑外科'] 颅咽管瘤
['脑外科'] 脑囊虫病
['神经内科', '脑外科'] 化脓性脑膜炎
['精神科', '神经内科'] 癫痫伴发的精神障碍
['神经内科'] 痴呆
['脑外科', '神经内科'] 小儿脑积水
['神经内科'] 难治性癫痫
['内分泌科'] 生长激素缺乏症
['神经内科', '脑外科'] 原发性癫痫
['妇科', '中医科'] 闭经性头痛
['儿科', '脑外科'] 新生儿胆红素脑病
['神经内科', '老年科'] 老年性癫痫
['急诊科'] 硫化氢中毒
['脑外科'] 脑病
['神经内科'] 酒精性脑萎缩
['脑外科', '肿瘤科'] 神经元肿瘤和神经元与神经胶质混合性肿瘤
['神经内科'] 戒酒综合征
['脑外科', '外伤科'] 头皮血肿
['神经内科', '儿科'] 新生儿化脓性脑膜炎
['神经内科', '脑外科'] 顶叶癫痫
['神经内科', '儿科'] CM1神经节苷脂贮积症
['肿瘤科', '普外科'] 神经胶质瘤
['神经内科'] 松果体瘤
['内分泌科'] 功能性下丘脑性闭经
['肿瘤科'] 基底细胞腺瘤
['脑外科'] 脑瘫
['神经内科'] 神经性头痛
['神经内科', '耳鼻喉科'] 内耳膜迷路积水
['神经内科'] 帕金森病性痴呆
['神经内科'] 小舞蹈病
['神经内科'] 疯牛病
['脑外科', '儿科'] 早期婴儿型癫痫性脑病
['神经内科', '脑外科'] 外伤后头痛
['神经内科'] 路易体痴呆
['神经内科', '儿科'] 中枢神经系统感染
['脑外科', '肿瘤科'] 胶质母细胞瘤
['神经内科

['呼吸内科', '心胸外科'] 纵隔囊肿
['心胸外科', '普外科'] 先天性膈疝
['心胸外科', '肿瘤科'] 原发性纵隔肿瘤
['心胸外科'] 纵膈感染
['心胸外科', '肿瘤科'] 纵隔囊肿及肿瘤
['呼吸内科'] 急性纵隔炎
['心胸外科', '肿瘤科'] 纵隔间叶源性肿瘤及其他肿瘤
['心胸外科'] 纵隔脓肿
['心胸外科', '肿瘤科'] 纵隔霍奇金淋巴瘤
['心胸外科', '肿瘤科'] 纵隔畸胎瘤
['心胸外科', '肿瘤科'] 植物神经系统肿瘤
['内分泌科', '儿科'] 小儿早老症
['脑外科', '急诊科'] 外伤后脑膨出
['脑外科', '肿瘤科'] 副肿瘤性边缘系统脑炎
['脑外科', '肿瘤科'] 枕骨大孔脑膜瘤
['传染科', '神经内科'] 脑型肺吸虫病
['脑外科', '肿瘤科'] 顶叶肿瘤
['神经内科', '儿科'] 小儿肺炎链球菌脑膜炎
['神经内科', '脑外科'] 小儿脑白质海绵状变性综合征
['神经内科'] 朊毒体病
['神经内科', '儿科'] 小儿发作性睡病
['神经内科', '儿科'] 新生儿反应低下
['儿科', '神经内科'] 小儿变形性肌张力障碍
['传染科'] 弓形体脑病
['肿瘤科'] 枕叶肿瘤
['神经内科', '儿科'] 小儿嗜血性流行性感冒杆菌脑膜炎
['普外科', '血管外科'] 上矢状窦血栓性静脉炎
['内分泌科', '儿科'] 小儿肢端肥大症和垂体性巨人症
['神经内科', '脑外科'] 外伤后脑脂肪栓塞
['脑外科', '肿瘤科'] 岩骨斜坡脑膜瘤
['脑外科', '神经内科'] 脑真菌性肉芽肿
['肿瘤科', '儿科'] 小儿血管网状细胞瘤
['神经内科', '传染科'] 东方马型脑炎
['内分泌科'] 特发性青春期延迟
['神经内科', '传染科'] 原发性阿米巴脑膜脑炎
['肿瘤科', '眼科'] 睑板腺癌
['眼科'] 慢性闭角型青光眼
['眼科'] 泪腺炎
['眼科'] 春季结膜炎
['眼科', '皮肤科'] 眼睑带状疱疹
['眼科', '性病科'] 梅毒性角膜病变
['眼科', '普外科'] 外伤性白内障
['眼科'] 急性虹膜睫状体炎
['眼科', '五官科'] 急性闭角型青光眼
['眼科', '中医科'] 视网膜中央动脉阻塞
['眼科'] 黄斑囊样水肿
['眼科', 

['眼科'] 化脓性眼内炎
['眼科'] 表层巩膜炎
['眼科'] 大泡性角膜病变
['眼科'] 细菌性角膜溃疡
['眼科'] 睑外翻
['眼科', '营养科'] 电脑视觉综合症
['眼科'] 浅层点状角膜炎
['眼科'] 新生血管性青光眼
['眼科'] 前葡萄膜炎
['眼科', '五官科'] 真菌性角膜溃疡
['神经内科', '眼科'] 压迫性视神经病变
['眼科'] 眼睑疖肿和脓肿
['眼科', '骨科'] 眼眶骨膜炎
['呼吸内科', '眼科'] 小儿疱疹性角膜结膜炎
['眼科'] Fuchs角膜内皮营养不良
['眼科', '心血管内科'] 恶性高血压视网膜病变
['外伤科', '眼科'] 晶体脱位
['眼科', '性病科'] 梅毒性葡萄膜炎
['眼科'] 变性性近视的眼底损害
['眼科'] 剥脱综合征
['眼科', '风湿科'] 结节性多动脉炎伴发的葡萄膜炎
['口腔科'] 眶下间隙感染
['眼科'] 眼眶蜂窝织炎
['肿瘤科', '呼吸内科'] 肺癌
['呼吸内科'] 慢性支气管炎
['呼吸内科', '心胸外科'] 肺气肿
['呼吸内科'] 支气管扩张
['呼吸内科', '传染科'] 支原体肺炎
['呼吸内科', '传染科'] 百日咳
['呼吸内科', '心胸外科'] 气胸
['风湿科', '中西医结合科'] 过敏性哮喘
['呼吸内科', '心胸外科'] 肺栓塞
['呼吸内科'] 阻塞性肺气肿
['呼吸内科'] 慢性咳嗽
['呼吸内科', '老年科'] 老年人肺炎
['呼吸内科'] 肺脓肿
['肿瘤科', '心胸外科'] 老年人肺癌
['呼吸内科'] 矽肺
['呼吸内科', '老年科'] 老年人肺气肿
['呼吸内科', '老年科'] 老年肺炎
['肿瘤科'] 小细胞肺癌
['儿科'] 新生儿呼吸窘迫综合征
['中医科', '呼吸内科'] 虚寒咳嗽
['呼吸内科', '心胸外科'] 肺大疱
['传染科'] 老年人肺结核病
['中医科', '呼吸内科'] 风热犯肺
['呼吸内科', '中医科'] 寒饮咳嗽
['呼吸内科', '心胸外科'] 肺真菌病
['呼吸内科', '心胸外科'] 肺炎性假瘤
['心胸外科', '呼吸内科'] 肺隔离症
['呼吸内科', '老年科'] 老年人特发性肺纤维化
['心胸外科', '儿科'] 先天性肺囊肿
['呼吸内科', '消化内

['口腔科', '普外科'] 乳头状囊腺癌
['口腔科'] 牙列拥挤
['口腔科'] 开合
['中医科'] 小儿流涎症
['口腔科', '肿瘤科'] 涎腺癌
['中医科'] 口糜
['口腔科', '肿瘤科'] 涎腺混合瘤
['口腔科', '普外科'] 深覆合
['口腔科'] 复发性坏死性黏膜周围炎
['口腔科'] 颊间隙感染
['口腔科'] 金属引起的口腔黏膜疾病
['口腔科', '皮肤科'] 黏膜白色海绵状痣
['口腔科'] 浆细胞性唇炎
['中医科'] 垂痈
['口腔科', '肿瘤科'] 涎腺的未分化癌
['中医科', '儿科'] 小儿疳渴
['口腔科', '肿瘤科'] 牙瘤
['儿科', '口腔科'] 小儿坏死性龈口炎
['口腔科'] 龅牙
['口腔科'] 滤泡性咽炎
[] 牙龈瘤
['耳鼻喉科'] 慢性咽炎
['儿科', '呼吸内科'] 小儿感冒
['肿瘤科', '耳鼻喉科'] 喉癌
['口腔科', '儿科'] 咽炎
['耳鼻喉科', '五官科'] 疱疹性咽峡炎
['内分泌科', '耳鼻喉科'] 腺样体肥大
['儿科', '呼吸内科'] 小儿流行性感冒
['传染科', '耳鼻喉科'] 白喉
['耳鼻喉科', '呼吸内科'] 病毒性咽炎
['耳鼻喉科'] 咽鼓管阻塞
['耳鼻喉科'] 咽异感症
['耳鼻喉科'] 咽部异物
['性病科'] 淋菌性咽炎
['肿瘤科', '心胸外科'] 肺腺癌
['呼吸内科'] 特发性肺纤维化
['呼吸内科', '普内科'] 金黄色葡萄球菌肺炎
['呼吸内科'] 衣原体肺炎
['儿科', '呼吸内科'] 小儿喘息样支气管炎
['心胸外科', '肿瘤科'] 肺转移瘤
['老年科', '心胸外科'] 老年人胸腔积液与胸膜炎
['中医科', '呼吸内科'] 肺咳
['心胸外科', '肿瘤科'] 肺部少见的恶性肿瘤
['呼吸内科', '传染科'] Q热肺炎
['呼吸内科', '儿科'] 呼吸道合胞病毒肺炎
['呼吸内科', '心血管内科'] 慢性呼吸衰竭
['儿科'] 乳汁吸入性肺炎
['呼吸内科', '普内科'] 腺病毒肺炎
['中医科', '呼吸内科'] 燥邪犯肺
['呼吸内科'] 急性细支气管炎
['儿科', '呼吸内科'] 小儿流行性喘憋性肺炎
['呼吸内科', '急诊科'] 高原肺水肿
['中医科', '产科'] 产后喘促
['呼

['消化内科', '胃肠外科'] 胆汁返流性胃炎
['消化内科'] 胃炎
['消化内科'] 慢性浅表性胃炎
['消化内科', '胃肠外科'] 消化性溃疡
['消化内科'] 急性胃炎
['消化内科'] 胃下垂
['胃肠外科', '消化内科'] 胃息肉
['消化内科', '普外科'] 胃、十二指肠溃疡出血
['消化内科', '胃肠外科'] 幽门梗阻
['消化内科'] 慢性糜烂性胃炎
['消化内科', '心胸外科'] 贲门失弛缓症
['胃肠外科'] 胃穿孔
['消化内科'] 吸收不良综合征
['产科', '妇产科'] 孕吐
['消化内科'] 急性糜烂性胃炎
['消化内科', '儿科'] 小儿幽门螺杆菌感染
['消化内科'] 胃潴留
['肿瘤科', '普外科'] 胃肠道间质瘤
['中医科'] 小儿食积
['胃肠外科', '儿科'] 先天性肥厚性幽门狭窄
['普外科'] 胃损伤
['中医科'] 痞满
['胃肠外科', '肿瘤科'] 胃类癌瘤
['消化内科', '普内科'] 倾倒综合征
['消化内科', '肝胆外科'] 非寄生虫性肝囊肿
['肝胆外科', '儿科'] 先天性肝内胆管囊状扩张症
['肝胆外科', '肿瘤科'] 原发性肝脂肪肉瘤
['肝胆外科'] 肝淀粉样变性
['儿科', '肝胆外科'] 小儿肝硬化
['中医科'] 肥气
['肝炎科', '儿科'] 小儿丙型病毒性肝炎
['消化内科', '普内科'] 缺血性肝炎
['中医科', '肝病科'] 肝厥
['肝胆外科', '肿瘤科'] 小儿肝肿瘤
['儿科', '肝胆外科'] 小儿先天性肝囊肿
['消化内科', '普内科'] 特发性非硬化性门脉高压综合征
['消化内科', '传染科'] 多房棘球蚴病
['消化内科', '肝胆外科'] 浓缩胆汁综合征
['消化内科', '肝炎科'] 小儿戊型病毒性肝炎
['肝胆外科', '消化内科'] 妊娠合并肝硬化
['肝胆外科', '肿瘤科'] 胆道华支睾吸虫病
['肝炎科', '儿科'] 小儿丁型病毒性肝炎
['儿科', '肝胆外科'] 小儿动脉肝脏发育异常综合征
['儿科', '传染科'] 小儿片吸虫病
['肝炎科'] 自身免疫性肝病
['消化内科'] 自身免疫性肝炎
['肝胆外科'] 肝内钙化灶
['肝炎科'] 布-加综合征
['胃肠外科', '普外科'] 急性阑尾炎
['消化内

...

['风湿科'] 二水焦磷酸钙结晶沉积症
['内分泌科', '骨科'] 褐黄病
['骨科', '中医科'] 慢性创伤性滑囊炎
['血液科', '中西医结合科'] 镰状细胞贫血
['儿科', '骨科'] 小儿关节过度活动综合征
['儿科', '骨科'] 小儿急性化脓性关节炎
['骨科'] 颈椎双侧关节突关节脱位
['骨科', '中医科'] 非淋球菌性细菌性关节炎
['骨科', '中西医结合科'] 褐黄病性关节炎
['骨科', '中西医结合科'] 松毛虫性骨关节炎
['骨科'] 松毛虫性骨关节病
['骨科', '针灸科'] 上干型胸廓出口综合征
['内分泌科', '儿科'] 黏多糖贮积症Ⅲ型
['儿科', '神经内科'] 小儿关节弯曲综合征
['风湿科', '骨科'] 卡普兰综合征
['骨科', '中医科'] 环跳疽
['中医科'] 顽痹
['骨科'] 透析性骨关节病
['骨科', '中西医结合科'] 淀粉样变病和淀粉样关节病
['血液科', '骨科'] 其他晶体性关节病
['骨科', '神经内科'] 颈椎病
['骨科', '中医科'] 腰椎间盘突出
['骨科', '风湿科'] 脊柱炎
['骨科'] 第三腰椎横突综合征
['骨科', '中医科'] 慢性腰背痛
['骨科'] 脊椎骨质增生
['骨科'] 脊柱侧凸
['骨科'] 急性颈椎间盘突出症
['骨科', '传染科'] 脊椎结核
['骨科'] 脊椎病
['骨科', '肿瘤科'] 椎管内神经鞘瘤
['骨科'] 颈腰综合征
['骨科'] 棘间韧带损伤
['骨科'] 颈椎结核
['骨科'] 颈椎后纵韧带骨化症
['骨科', '针灸科'] 棘上韧带损伤
['骨科', '急诊科'] 胸、腰椎椎体单纯性、楔形压缩骨折
['骨科'] 胸椎结核椎旁脓肿穿入胸腔
['骨科'] 脊椎椎弓峡部裂
['骨科'] 第3腰椎横突过长畸形
['骨科', '中西医结合科'] 特发性脊柱侧凸
['骨科', '急诊科'] 胸、腰椎爆裂骨折
['口腔科'] 齿状突发育畸形
['骨科', '肿瘤科'] 椎管内脊膜瘤
['骨科', '中西医结合科'] 脊椎骨骨髓炎
['骨科'] 棘突压痛
['骨科'] 棘突骨折
['骨科'] 先天性脊柱侧凸
['肿瘤科', '骨科'] 原发性椎管内肿瘤
['骨科'] 颈椎后脱位
['骨科'] 颈椎黄韧带骨化症
['骨科'] 颈椎

['泌尿外科'] 膀胱输尿管反流
['肾内科', '风湿科'] 类风湿性关节炎的肾损害
['肾内科', '皮肤科'] 小儿过敏性紫癜肾炎
['肾内科', '骨科'] 指甲-髌骨综合征
['肾内科'] 慢性反流性肾病
['儿科', '骨科'] 小儿指甲-髌骨综合征
['儿科', '泌尿外科'] 小儿尿石症
['传染科'] 钩端螺旋体病的肾损害
['普外科', '皮肤科'] 冷球蛋白血症肾损害
['肾内科', '血液科'] 原发性巨球蛋白血症肾损害
['泌尿外科'] 尿瘘
['泌尿外科', '普外科'] 梗阻性尿路疾病
['儿科', '泌尿外科'] 小儿尿路梗阻
['皮肤科', '妇科'] 鲍温样丘疹病
['风湿科', '产科'] 妊娠合并类风湿关节炎
['妇科', '皮肤科'] 外阴天疱疮
['皮肤科', '中医科'] 脚气
['皮肤科'] 伊藤痣
['风湿科', '老年科'] 老年人痛风
['泌尿外科'] 女性尿瘘
['泌尿外科', '妇科'] 间质性膀胱炎、局限性外阴炎和脱屑性阴道炎综合征
['妇科', '肿瘤科'] 外阴神经纤维瘤
['风湿科', '中医科'] 嗜酸性粒细胞增多-肌痛综合征
['皮肤科'] 脚气病
['儿科', '消化内科'] 小儿过敏性紫癜
['外伤科', '普外科'] 非冻结性冷伤
['神经内科', '普内科'] 妊娠合并重症肌无力
['内分泌科', '皮肤科'] 胫前黏液水肿
['血液科', '中西医结合科'] 进行性色素性紫癜性皮病
['皮肤科'] 足跟瘀斑
['儿科', '血液科'] 小儿变应性亚败血症综合征
['血液科', '肿瘤科'] 急性杂合性白血病
['血液科'] 成人型慢性粒细胞白血病
['血液科', '风湿科'] 原发性混合型冷球蛋白血症性血管炎
['儿科', '血液科'] 小儿维斯科特-奥尔德里奇综合征
['皮肤科'] 疼痛性脂肪疝
['肛肠外科', '皮肤科'] 藏毛性疾病
['血管外科'] 小腿红绀病
['儿科', '血液科'] 小儿血友病
['皮肤科'] 急性脂肪皮肤硬化症
['骨科', '血管外科'] 腘动脉损伤
['皮肤科'] 遗传性硬化性皮肤异色症
['儿科', '皮肤科'] 小儿血管角质瘤综合征
['皮肤科', '风湿科'] 痛性青紫综合征
['血液科', '肿瘤科'] 淋巴瘤细胞白血病
['血液科', '肿

['眼科', '风湿科'] 强直性脊柱炎性巩膜炎
['眼科'] 视网膜动脉阻塞
['眼科'] 幼年性黄色肉芽肿
['眼科', '肾内科'] 肾小管间质性肾炎葡萄膜炎综合征
['眼科', '风湿科'] 系统性红斑狼疮伴发的葡萄膜炎
['儿科', '呼吸内科'] 小儿毛细支气管炎
['呼吸内科', '传染科'] HIV相关呼吸道感染
['呼吸内科', '老年科'] 老年人支气管哮喘
['肿瘤科'] 肺癌皮肤转移
['呼吸内科', '心胸外科'] 气管、支气管狭窄
['肿瘤科', '呼吸内科'] Pancoast综合征
['呼吸内科'] Hughes-Stovin综合征
['呼吸内科', '肾内科'] 肺出血－肾炎综合征
['呼吸内科'] 非结核性分枝杆菌病
['肝胆外科', '肝病科'] 肝肺综合症
['呼吸内科'] 急性肺源性心脏病
['耳鼻喉科', '传染科'] 虫霉病
['肿瘤科', '耳鼻喉科'] 原发性鼻腔淋巴瘤
['耳鼻喉科', '皮肤科'] 致命性中线肉芽肿
['皮肤科'] 酒渣鼻样结核疹
['皮肤科'] 耳湿疹
['呼吸内科', '传染科'] 卡他莫拉菌感染
['耳鼻喉科'] 颞骨岩部炎
['五官科', '皮肤科'] 颜面部疖痈
['骨科', '中医科'] 颞骨和下颔骨关节异常
['皮肤科'] 幼年黑素瘤
['儿科'] 小儿面部偏侧肥大综合征
['传染科', '性病科'] 非性病性梅毒
['血液科', '儿科'] 新生儿红细胞增多症-高黏滞度综合征
['眼科'] 良性淋巴上皮病
['眼科', '肿瘤科'] 眼眶静脉性血管瘤
['眼科', '五官科'] 眶底骨折
['儿科', '眼科'] 小儿眼外肌麻痹-视网膜色素变性-心脏传导阻滞综合征
['消化内科', '老年科'] 老年人缺血性结肠炎
['消化内科', '胃肠外科'] 溃疡病大出血
['普内科'] 肠白塞病
['消化内科', '传染科'] 小肠结肠炎耶尔森菌肠炎
['消化内科', '老年科'] 老年人消化性溃疡
['消化内科', '传染科'] 轮状病毒所致胃肠炎
['骨科', '中医科'] 肠道感染后反应性关节炎
['消化内科', '传染科'] 贾第虫病
['消化内科', '传染科'] 日本血吸虫病
['口腔科'] 颞下颌关节脱位
['男科'] 急性腮腺炎性睾丸炎
['呼吸内科', '口腔科'] 口腔

['儿科', '血液科'] 小儿继发性血小板增多症
['内分泌科', '风湿科'] 尿黑酸尿与褐黄病
['血管外科'] 血栓性浅静脉炎
['血管外科'] 原发性下肢静脉瓣膜关闭不全
['血液科', '肿瘤科'] 老年人慢性淋巴细胞白血病
['风湿科', '中西医结合科'] 变应性皮肤结节性血管炎
['血液科', '老年科'] 老年人免疫性血小板减少性紫癜
['血液科', '肿瘤科'] 恶性组织细胞病
['儿科', '血液科'] 小儿特发性血小板减少性紫癜
['传染科'] 阴囊丝虫病
['血管外科'] 腘动脉瘤
['血液科', '肿瘤科'] 老年人慢性粒细胞白血病
['心理咨询科', '皮肤科'] 神经官能性表皮剥脱
['皮肤科'] 松毛虫皮炎
['血管外科'] 先天性静脉畸形肢体肥大综合征
['传染科', '普内科'] 基孔肯雅热
['神经内科'] 脯肽酶缺乏症
['传染科'] 病毒性出血热
['血液科', '中西医结合科'] 淤滞性紫癜
['血液科', '皮肤科'] 皮肤白血病
['肿瘤科', '皮肤科'] 皮肤淋巴细胞瘤
['皮肤科'] 枕骨下硬结性毛囊炎
['皮肤科'] 泛发性扁平黄色瘤
['血液科', '妇科'] 妊娠合并白血病
['普外科', '血管外科'] 小儿遗传性血管神经性水肿
['肾内科'] 嗜酸性粒细胞性膀胱炎
['风湿科', '中西医结合科'] 复发性皮肤坏死性嗜酸粒细胞血管炎
['风湿科'] 微病毒B19感染性风湿病
['儿科', '血液科'] 小儿病毒相关吞噬血细胞综合征
['血液科', '皮肤科'] 蕈样真菌病和Sezary综合征
['传染科'] 内脏血吸虫炎
['皮肤科'] 链球菌坏死
['呼吸内科', '消化内科'] 链杆菌病
['骨科', '中医科'] 寄生虫性关节炎
['普内科', '儿科'] 小儿埃莱尔-当洛综合征
['皮肤科', '儿科'] 小儿混合性结缔组织病
['皮肤科'] 瘰疬分枝杆菌感染
['皮肤科'] 立克次体痘
['皮肤科', '儿科'] 小儿播散性脂肪肉芽肿综合征
['血液科'] 维斯科特-奥尔德里奇综合征
['血液科', '皮肤科'] 塞泽里综合征
['肿瘤科'] 继发性皮肤CD30阳性大细胞淋巴瘤
['肿瘤科'] 获得性免疫缺陷综合征相关淋巴瘤
['血液科'] 再生障碍性贫血
['心血管内科', '血管外科'

['中医科', '儿科'] 疳
['儿科', '营养科'] 小儿营养不良
['儿科', '营养科'] 小儿营养不良性水肿
['消化内科', '神经内科'] 福尔马林中毒
['中医科'] 阴冷
['中医科'] 寒滞肝脉证
['中医科', '心血管内科'] 胸痹
['中医科', '心血管内科'] 胸痹心痛
['中医科'] 产后血虚热
['中医科'] 胃寒呕吐
['中医科'] 气虚眩晕
['产科', '急诊科'] 产后出血
['消化内科', '急诊科'] 驱蛔灵中毒
['传染科'] 克里米亚-刚果出血热
['妇科'] 更年期综合征
['产科', '中医科'] 产褥中暑
['内分泌科'] 非甲状腺性的病态综合征
['内分泌科', '儿科'] 糖原贮积病
['中医科'] 干血痨
['内分泌科'] 异位ACTH综合征
['儿科', '普内科'] 小儿急性间歇性卟啉病
['中医科'] 内热
['急诊科'] 河豚毒素中毒
['内分泌科', '营养科'] 蛋白质-能量营养不良
['口腔科', '儿科'] 小儿错畸形
['肿瘤科'] 小儿横纹肌肉瘤
['肿瘤科', '骨科'] 骨化性纤维瘤
['儿科', '普内科'] 小儿甲基丙二酸血症
['内分泌科', '皮肤科'] 医源性多毛症
['中医科'] 妊娠热病
['骨科'] 软骨外胚层发育不全
['肿瘤科'] 小儿结节病
['中医科'] 热汗
['传染科', '肾内科'] 流行性出血热
['中医科'] 痿病
['儿科'] 新生儿发热
['中医科'] 太阳病
['妇科'] 急性盆腔炎
['急诊科'] 钡中毒
['口腔科', '骨科'] 氟骨症
['中医科'] 阳虚发热
['普内科', '内分泌科'] 黏多糖贮积症
['中医科'] 走哺
['呼吸内科', '急诊科'] 铍中毒
['急诊科'] 锑中毒
['中医科'] 血虚头痛
['儿科', '营养科'] 小儿蛋白质-能量营养不良
['中医科'] 三痹
['儿科', '营养科'] 坏血病
['中医科', '呼吸内科'] 痰火
['中医科'] 恶寒发热
['中医科'] 湿热病
['普外科', '普内科'] 急性脓肿
['儿科', '血液科'] 小儿家族性低血磷性佝偻病
['消化内科', '外伤科'] 强碱类中毒
['急诊科', '消化内科'] 白果中毒
['中医科', '呼吸内科'] 痰热咳嗽
[

In [41]:
'ss'.endswith('科')

False

In [45]:
df1.shape

(6483, 2)